# Examen Projet NLP TALA540A 

## Auteurs :
- AROUN Jeevya
- NGAUV Nicolas
- THEZENAS Anissa

## I) Présentation, visualisation, nettoyage/réorganisation et analyse du dataset

### 1. Présentation du dataset

- Utilisation de Kaggle dans le projet

Nous avions initialement envisagé de constituer notre propre jeu de données en extrayant, via un script de web scraping, les informations qui nous intéressaient sur le site Fragrantica. Malheureusement, nous n’avons pas pu accéder aux commentaires des utilisateurs, une composante essentielle pour notre projet d’analyse. Après plusieurs tentatives infructueuses pour récupérer ces données, nous avons finalement décidé d’utiliser un jeu de données déjà disponible, que nous avons trouvé sur Kaggle. Voici le lien vers ce dataset : [Fragrantica Data sur Kaggle](https://www.kaggle.com/datasets/joehusseinmama/fragrantica-data).

- Importance de Kaggle pour le NLP

Kaggle est une plateforme précieuse pour la communauté de l’apprentissage automatique et du traitement du langage naturel (NLP), car elle offre un vaste choix de jeux de données, des ressources et des tutoriels partagés par des praticiens du monde entier. Pour des projets NLP, Kaggle fournit souvent des ensembles de données annotés et prêts à l’emploi, permettant ainsi de se concentrer sur l’entraînement et l’optimisation des modèles sans passer par une phase de collecte et de prétraitement longue (mais nous allons tout de même effectuer une réorganisation et un nettoyage de notre dataset pour faciliter la suite de notre travail).

- Jeu de données Fragrantica

Le jeu de données Fragrantica utilisé ici contient des descriptions et caractéristiques de parfums, ce qui offre une base solide pour notre analyse de texte, essentielle dans notre projet de NLP.

- Avantages de l'utilisation de Kaggle

Grâce à cette ressource, nous pouvons ainsi avancer efficacement dans nos expérimentations et analyses sans les contraintes liées au scraping et à la collecte de données directement depuis des sites comme Fragrantica.

### 2. Visualisation du dataset

In [1]:
import pandas as pd

def charger_dataset(fichier_csv):
    # Charger le fichier CSV dans un DataFrame
    df = pd.read_csv(fichier_csv)

    # Observer les 4 premières lignes
    print("Voici les 4 premières lignes du dataset :\n")
    print(df.head())
    print("="*100)

    # Vérification des colonnes vides ou des données manquantes
    print("\nInformations sur le dataset :\n")
    print(df.info())
    print("="*100)

    return df


# Définir le chemin du fichier CSV
fichier_csv = '../data/perfumes_table.csv'  

# Charger et nettoyer le dataset
dataset = charger_dataset(fichier_csv)


Voici les 4 premières lignes du dataset :

   rating                                              notes  \
0    5.00  ['Vanila', 'Madagascar Vanilla', 'Vanilla Abso...   
1    4.22  ['Mint', 'Lavender', 'Nutmeg', 'elemi', 'Cypre...   
2    3.81              ['Neroli', 'Vetiver', 'Basil', 'Fig']   
3    4.00  ['Orange Blossom', 'Star Anise', 'Pear', 'Rose...   
4    4.22  ['Petitgrain', 'Orange', 'Bergamot', 'Lemon Ve...   

                                    designer  \
0             fiorucci perfumes and colognes   
1        maiora parfum perfumes and colognes   
2             guerlain perfumes and colognes   
3   jean paul gaultier perfumes and colognes   
4  santa maria novella perfumes and colognes   

                                             reviews  \
0                                                 []   
1                                                 []   
2  ["the opening is too sharp that it reminds me ...   
3  ['I must ask. What part of the country did thi...   
4  

### 3. Nettoyage/Réorganisation/Prétraitement du dataset
- Cela nous arrangerait pour la suite d'avoir pour chaque parfum son nom et le genre du public visé par celui-ci : on va donc créer ces colonnes
- On va aussi nettoyer le colonne 'designer' qui contient de l'information superflue sur pratiquement toutes les lignes (on veut juste le nom de la marque, mais on a en plus " perfumes an colognes") :  toujours la même info superflue, donc ne discrimine pas, donc pas intéressant, donc on enlève

In [2]:
import pandas as pd
import re  

# Charger le fichier CSV
df = pd.read_csv('../data/perfumes_table.csv')

# Fonction pour extraire le nom du parfum et le genre depuis 'title'
def extraire_nom_et_genre(title):
    if isinstance(title, str):  # Vérifie si 'title' est une chaîne de caractères
        if " for women and men" in title:
            return title.replace(" for women and men", ""), "for women and men"
        elif " for women" in title:
            return title.replace(" for women", ""), "for women"
        elif " for men" in title:
            return title.replace(" for men", ""), "for men"
        else:
            return title, "unknown"  # Genre inconnu si non précisé
    else:
        return title, "unknown"  # Valeurs par défaut pour les titres NaN

# Appliquer la fonction et renseigner les nouvelles colonnes
df[['name', 'genre']] = df['title'].apply(lambda x: pd.Series(extraire_nom_et_genre(x)))

# Supprimer " perfumes and colognes" à la fin de la colonne 'designer' si présent
df['designer'] = df['designer'].str.replace(" perfumes and colognes$", "", regex=True)

# Supprimer, pour chaque ligne de 'name', la chaîne de caractères de 'designer' sans tenir compte de la casse
df['name'] = df.apply(lambda row: re.sub(re.escape(str(row['designer'])), "", str(row['name']), flags=re.IGNORECASE).strip() if pd.notnull(row['designer']) else row['name'], axis=1)

# Réorganiser les colonnes en mettant 'name' et 'genre' en première et deuxième position
colonnes_ordre = ['name', 'genre'] + [col for col in df.columns if col not in ['name', 'genre']]
df = df[colonnes_ordre]

# Sauvegarder le DataFrame modifié dans un nouveau fichier CSV
df.to_csv('../data/cleaned_perfumes_table.csv', index=False)

print("Les colonnes 'name' et 'genre' ont été ajoutées et déplacées en premières positions, 'designer' a été nettoyée, et la chaîne de 'designer' a été supprimée de 'name' sans tenir compte de la casse.")


Les colonnes 'name' et 'genre' ont été ajoutées et déplacées en premières positions, 'designer' a été nettoyée, et la chaîne de 'designer' a été supprimée de 'name' sans tenir compte de la casse.


### 4) Description du dataset

- Type de données, taille, prétraitement effectué (ex. : tokenisation, lemmatisation, suppression des stopwords, etc.)
    - Type de données : 
    - Taille : 690,8 Mio avant prétraitement et 691,2 Mio après
    - Prétraitement effectué :

- Description statistique (distribution des classes, fréquence des mots, longueur moyenne des commentaires, etc.) et qualitative (exemples typiques, difficultés linguistiques, caractéristiques particulières) : juste en dessous

### À partir de maintenant, on va donc utiliser le nouveau dataset contenu dans le fichier '../data/cleaned_perfumes_table.csv' pour la suite du projet

## II) Présentation de l'outil

## III) Mise en oeuvre et évaluation

## IV) Discussion et analyse